Membres du groupe 
- AFOMALE David
- DOGBO Sarah
- BOTRE Aboudou
- TEPE Paulin
- NOYOULIWA Victoire

### Contexte général
Préparation des données ASR en langues Éwé et Gegbe

Ce notebook décrit et exécute l'ensemble des étapes nécessaires
pour transformer les données brutes collectées (audio + texte)
en un dataset utilisable pour l'entraînement ou l'évaluation
d'un modèle de reconnaissance automatique de la parole (ASR).


In [1]:
import sys
from pathlib import Path
if str(Path.cwd().parent) not in sys.path:
    sys.path.append(str(Path.cwd().parent))


**Vérification de la structure du projet : s’assurer que les données RAW existent avant le processing.**

In [2]:
from src.config.settings import EWE_RAW_DIR, GEGBE_RAW_DIR

ewe_audio = EWE_RAW_DIR / "audio"
gegbe_audio = GEGBE_RAW_DIR / "audio"

print("Audio RAW Éwé :", len(list(ewe_audio.glob("*.mp3"))) if ewe_audio.exists() else 0)
print("Audio RAW Gegbe :", len(list(gegbe_audio.glob("*.mp3"))) if gegbe_audio.exists() else 0)
print("Métadonnées Éwé :", (EWE_RAW_DIR / "metadata" / "ewe_bible_raw.json").exists())
print("Métadonnées Gegbe :", (GEGBE_RAW_DIR / "metadata" / "gegbe_bible_raw.json").exists())

Audio RAW Éwé : 413
Audio RAW Gegbe : 388
Métadonnées Éwé : True
Métadonnées Gegbe : True


### Étape 1 : Processing audio

Objectif : Standardiser tous les fichiers audio (WAV, mono, 16 kHz).

In [3]:
from src.preprocessing.audio_processing import convert_mp3_to_wav_16k

print("Conversion des audios Éwé...")
convert_mp3_to_wav_16k(lang="ewe")

print("Conversion des audios Gegbe...")
convert_mp3_to_wav_16k(lang="gegbe")

INFO:src.preprocessing.audio_processing:413 fichiers audio trouvés au total


Conversion des audios Éwé...


INFO:src.preprocessing.audio_processing:✔ 1ch_01.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_02.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_03.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_04.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_05.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_06.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_07.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_08.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_09.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_10.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_11.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_12.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_13.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_14.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_15.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_16.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_17.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_18.wav
INFO:src.preprocessing.audio_processing:✔ 1ch_

Conversion des audios Gegbe...


### Étape 2 : Nettoyage des transcriptions

In [ ]:
from src.preprocessing.text_cleaning import clean_all_texts

print("Nettoyage des textes...")
clean_all_texts()

Nettoyage des textes...


INFO:src.preprocessing.text_cleaning:✔ 1ch_01_1.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_10.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_11.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_12.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_13.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_14.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_15.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_16.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_17.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_18.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_19.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_2.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_20.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_21.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_22.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_23.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_24.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_25.txt
INFO:src.preprocessing.text_cleaning:✔ 1ch_01_26

### Étape 3 : Construction du dataset ASR final

In [6]:
from src.preprocessing.dataset_builder import build_asr_dataset

print("Génération du fichier CSV pour l'entraînement...")
build_asr_dataset()

Génération du fichier CSV pour l'entraînement...


### Inspection du dataset final

In [7]:
import pandas as pd
from src.config.settings import PROCESSED_DIR

dataset_path = PROCESSED_DIR / "bible_asr_dataset.csv"
if dataset_path.exists():
    df = pd.read_csv(dataset_path)
    print(f"Dataset chargé avec {len(df)} entrées.")
    display(df.head())
else:
    print("Dataset non trouvé.")

Dataset non trouvé.
